In [ ]:
!pip -q install -U "tabpfn == 2.2.1"
!pip -q install pytorch-tabnet

# ============================================================
# CELL 1: IMPORTS & CONFIGURATION
# ============================================================

import os, random, gc, warnings
warnings.filterwarnings("ignore")

SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["TF_DETERMINISTIC_OPS"] = "1"

random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd
import joblib

import tensorflow as tf
tf.random.set_seed(SEED)
try:
    tf.config.experimental.enable_op_determinism()
except Exception:
    pass

from tensorflow import keras
from tensorflow.keras.layers import (
    Input, Dense, GRU, Bidirectional, Layer,
    Concatenate, Dropout, SpatialDropout1D, BatchNormalization,
    GaussianNoise, Masking
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pathlib import Path
from tqdm import tqdm

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.optimize import minimize_scalar

import xgboost as xgb
from catboost import CatBoostClassifier
from tabpfn import TabPFNClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import optuna
from optuna.samplers import TPESampler

keras.backend.clear_session()
gc.collect()

print("TF:", tf.__version__, "| PyTorch:", torch.__version__, "| SEED =", SEED)

DATA_PATH = Path("/kaggle/input/project/mallorn-astronomical-classification-challenge")

BANDS = ["u", "g", "r", "i", "z", "y"]
BAND_MAP = {b: i for i, b in enumerate(BANDS)}

MAX_SEQ_LEN = 300
N_FEATURES_PER_STEP = 4

MODEL_DIR = "saved_models"
os.makedirs(MODEL_DIR, exist_ok=True)

GBM_MODEL_DIR = "/kaggle/working/models"
os.makedirs(GBM_MODEL_DIR, exist_ok=True)

FEAT_TRAIN_PKL = "/kaggle/input/2d-gp-features/kaggle/working/cache/train_features_2dgp_gpy.pkl"
FEAT_TEST_PKL  = "/kaggle/input/2d-gp-features/kaggle/working/cache/test_features_2dgp_gpy.pkl"

N_FOLDS = 5
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

# SelectKBest configuration (cho cả NN meta và GBM)
USE_SELECTKBEST = False   # Set False to disable feature selection
K_BEST_RATIO = 0.8       # Keep 80% of features (or use absolute number if < 1)
K_BEST_MIN = 200         # Minimum number of features to keep
K_BEST_MAX = None        # Maximum number of features (None = no limit)

# Device for PyTorch models
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"PyTorch models will use device: {DEVICE}")

print("Config done!")


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# ============================================================
# CELL 2: LOAD DATA
# ============================================================

print("\n--- Loading Data ---")

train_log = pd.read_csv(DATA_PATH / "train_log.csv")
test_log  = pd.read_csv(DATA_PATH / "test_log.csv")

def load_lc(split, kind):
    return pd.read_csv(DATA_PATH / split / f"{kind}_full_lightcurves.csv")

train_lc = pd.concat([load_lc(s, "train") for s in train_log["split"].unique()], ignore_index=True)
test_lc  = pd.concat([load_lc(s, "test")  for s in test_log["split"].unique()],  ignore_index=True)

train_feat = pd.read_pickle(FEAT_TRAIN_PKL)
test_feat  = pd.read_pickle(FEAT_TEST_PKL)

y = train_log["target"].values.astype(np.int32)

print(f"Train LC: {train_lc.shape}, Test LC: {test_lc.shape}")
print(f"Train objects: {len(train_log)}, Test objects: {len(test_log)}")
print(f"TDE count: {train_log['target'].sum()}, TDE ratio: {train_log['target'].mean()*100:.2f}%")


--- Loading Data ---
Train LC: (479384, 5), Test LC: (1145125, 5)
Train objects: 3043, Test objects: 7135
TDE count: 148, TDE ratio: 4.86%


In [ ]:
# ============================================================
# CELL 9: FEATURE STATISTICS ANALYSIS
# ============================================================

print("\n" + "="*70)
print("ANALYSIS 1: FEATURE STATISTICS")
print("="*70)

# Analyze original features before scaling
print("\n1. NaN/Inf Analysis:")
nan_counts = pd.DataFrame({
    'feature': feature_names,
    'nan_count': [np.isnan(X_raw[:, i]).sum() for i in range(X_raw.shape[1])],
    'inf_count': [np.isinf(X_raw[:, i]).sum() for i in range(X_raw.shape[1])],
    'zero_count': [(X_raw[:, i] == 0).sum() for i in range(X_raw.shape[1])],
})

nan_counts['nan_pct'] = (nan_counts['nan_count'] / len(X_raw) * 100).round(2)
nan_counts['zero_pct'] = (nan_counts['zero_count'] / len(X_raw) * 100).round(2)

high_nan = nan_counts[nan_counts['nan_pct'] > 50].sort_values('nan_pct', ascending=False)
print(f"\n  Total features: {len(nan_counts)}")
print(f"  Features with >50% NaN: {len(high_nan)}")
if len(high_nan) > 0:
    print(f"  Top 10 features with most NaN:")
    print(high_nan.head(10)[['feature', 'nan_pct', 'zero_pct']].to_string(index=False))

print("\n2. Feature Scale Analysis:")
feature_stats = pd.DataFrame({
    'feature': feature_names,
    'mean': [np.nanmean(X_raw[:, i]) for i in range(X_raw.shape[1])],
    'std': [np.nanstd(X_raw[:, i]) for i in range(X_raw.shape[1])],
    'min': [np.nanmin(X_raw[:, i]) for i in range(X_raw.shape[1])],
    'max': [np.nanmax(X_raw[:, i]) for i in range(X_raw.shape[1])],
    'q25': [np.nanpercentile(X_raw[:, i], 25) for i in range(X_raw.shape[1])],
    'q75': [np.nanpercentile(X_raw[:, i], 75) for i in range(X_raw.shape[1])],
})

# Remove NaN/inf for analysis
feature_stats = feature_stats.replace([np.inf, -np.inf], np.nan)

print(f"  Mean std across features: {feature_stats['std'].mean():.4f}")
print(f"  Max std: {feature_stats['std'].max():.4f}")
print(f"  Min std: {feature_stats['std'].min():.4f}")

# Features with extreme scales
extreme_std = feature_stats[feature_stats['std'] > 1000].sort_values('std', ascending=False)
if len(extreme_std) > 0:
    print(f"\n  Features with VERY HIGH std (>1000): {len(extreme_std)}")
    print(extreme_std.head(10)[['feature', 'std', 'min', 'max']].to_string(index=False))

zero_var = feature_stats[feature_stats['std'] < 1e-6]
if len(zero_var) > 0:
    print(f"\n  Features with ZERO/NEAR-ZERO variance (<1e-6): {len(zero_var)}")
    print(zero_var[['feature', 'std']].head(20).to_string(index=False))

print("\n3. Feature Distribution Skewness:")
from scipy.stats import skew
feature_stats['skewness'] = [skew(X_raw[:, i]) if np.nanstd(X_raw[:, i]) > 1e-6 else 0 
                             for i in range(X_raw.shape[1])]
high_skew = feature_stats[feature_stats['skewness'].abs() > 5].sort_values('skewness', key=abs, ascending=False)
if len(high_skew) > 0:
    print(f"  Features with HIGH skewness (|skew| > 5): {len(high_skew)}")
    print(high_skew.head(10)[['feature', 'skewness', 'mean', 'std']].to_string(index=False))


ANALYSIS 1: FEATURE STATISTICS

1. NaN/Inf Analysis:

  Total features: 308
  Features with >50% NaN: 0

2. Feature Scale Analysis:
  Mean std across features: 3663.5524
  Max std: 1097813.4700
  Min std: 0.0000

  Features with VERY HIGH std (>1000): 7
               feature          std           min          max
         gp2d_ls_ratio 1.097813e+06  6.153001e-08 4.299914e+07
u_gp2d_integrated_flux 2.955390e+03 -3.252791e+04 8.786704e+04
g_gp2d_integrated_flux 2.624058e+03 -2.264464e+04 6.584307e+04
r_gp2d_integrated_flux 2.447970e+03 -1.457523e+04 4.652942e+04
i_gp2d_integrated_flux 2.395894e+03 -1.254920e+04 4.101660e+04
y_gp2d_integrated_flux 2.349733e+03 -1.171562e+04 3.981636e+04
z_gp2d_integrated_flux 2.348133e+03 -1.210663e+04 4.057562e+04

  Features with ZERO/NEAR-ZERO variance (<1e-6): 5
     feature  std
       Z_err  0.0
count_snr_-3  0.0
count_snr_-5  0.0
 frac_snr_-3  0.0
 frac_snr_-5  0.0

3. Feature Distribution Skewness:
  Features with HIGH skewness (|skew| > 5): 11

=> Apply Select Features

In [ ]:
# ============================================================
# CELL 7: PREPARE FEATURES WITH SelectKBest (FOR GBM MODELS)
# ============================================================
USE_SELECTKBEST = True
print("\n" + "="*60)
print("LOADING DATA (REUSED IN-MEMORY) FOR GBM")
print("="*60)
K_BEST_RATIO = 0.8
X_df = train_feat.copy()
X_test_df = test_feat.copy()

print(f"Train features: {X_df.shape}")
print(f"Test features: {X_test_df.shape}")
print(f"TDE count: {y.sum()}, TDE ratio: {y.mean()*100:.2f}%")

print("\n--- Preparing GBM Features ---")

if "object_id" in X_df.columns:
    X_df = X_df.drop(columns=["object_id"])
if "object_id" in X_test_df.columns:
    X_test_df = X_test_df.drop(columns=["object_id"])

# Deterministic order
common_cols = sorted(set(X_df.columns) & set(X_test_df.columns))
X_df = X_df[common_cols]
X_test_df = X_test_df[common_cols]

X_df = X_df.replace([np.inf, -np.inf], np.nan).fillna(0)
X_test_df = X_test_df.replace([np.inf, -np.inf], np.nan).fillna(0)

feature_names = X_df.columns.tolist()
X_raw = X_df.values
X_test_raw = X_test_df.values

n_features_original = X_raw.shape[1]
print(f"Original number of GBM features: {n_features_original}")

feature_selector = None
selected_feature_names = feature_names

os.makedirs(GBM_MODEL_DIR, exist_ok=True)

if USE_SELECTKBEST and n_features_original > K_BEST_MIN:
    # Calculate k_best
    if K_BEST_RATIO < 1.0:
        k_best = int(n_features_original * K_BEST_RATIO)
    else:
        k_best = int(K_BEST_RATIO)
    
    # Apply min/max constraints
    k_best = max(K_BEST_MIN, k_best)
    if K_BEST_MAX is not None:
        k_best = min(K_BEST_MAX, k_best)
    k_best = min(k_best, n_features_original)
    
    print(f"\nApplying SelectKBest (GBM): {n_features_original} -> {k_best} features")
    print(f"  (ratio={K_BEST_RATIO}, min={K_BEST_MIN}, max={K_BEST_MAX})")
    
    # Fit SelectKBest on full training data
    feature_selector = SelectKBest(score_func=f_classif, k=k_best)
    X = feature_selector.fit_transform(X_raw, y)
    X_test = feature_selector.transform(X_test_raw)
    
    # Get selected feature names
    selected_mask = feature_selector.get_support()
    selected_feature_names = [feature_names[i] for i in range(len(feature_names)) if selected_mask[i]]
    
    print(f"Selected {len(selected_feature_names)} features for GBM")
    
    # Save selector
    joblib.dump(feature_selector, f"{GBM_MODEL_DIR}/gbm_feature_selector.pkl")
    joblib.dump(selected_feature_names, f"{GBM_MODEL_DIR}/gbm_selected_feature_names.pkl")
    print(f"✅ Saved GBM feature selector and selected feature names")
    
else:
    print(f"\nSkipping SelectKBest for GBM (using all {n_features_original} features)")
    X = X_raw
    X_test = X_test_raw
    
    # Save all feature names
    joblib.dump(selected_feature_names, f"{GBM_MODEL_DIR}/gbm_all_feature_names.pkl")

print(f"\nFinal GBM feature count: {X.shape[1]}")
print(f"Final GBM train shape: {X.shape}, Final GBM test shape: {X_test.shape}")

scale_pos_weight = (y == 0).sum() / (y == 1).sum()
print(f"Scale pos weight: {scale_pos_weight:.2f}")


LOADING DATA (REUSED IN-MEMORY) FOR GBM
Train features: (3043, 309)
Test features: (7135, 309)
TDE count: 148, TDE ratio: 4.86%

--- Preparing GBM Features ---
Original number of GBM features: 308

Applying SelectKBest (GBM): 308 -> 246 features
  (ratio=0.8, min=200, max=None)
Selected 246 features for GBM
✅ Saved GBM feature selector and selected feature names

Final GBM feature count: 246
Final GBM train shape: (3043, 246), Final GBM test shape: (7135, 246)
Scale pos weight: 19.56


In [ ]:
# ============================================================
# CELL 8: TRAIN CATBOOST (WITH BEST-FOLD SUBMISSION)
# ============================================================
import numpy as np
import pandas as pd
import joblib
from itertools import combinations
from scipy.stats import rankdata
from scipy.optimize import minimize_scalar

print("\n" + "="*60)
print("TRAINING CATBOOST")
print("="*60)

cb_params = {
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "depth": 8,
    "learning_rate": 0.05,
    "iterations": 2000,
    "l2_leaf_reg": 3.0,
    "random_seed": SEED,
    "bootstrap_type": "Bernoulli",
    "subsample": 0.8,
    "colsample_bylevel": 0.8,
    "min_data_in_leaf": 20,
    "class_weights": [1.0, scale_pos_weight],
    "task_type": "CPU",
    "verbose": False,
}

cb_oof = np.zeros(len(y))
cb_test = np.zeros(len(X_test))
cb_models = []
cb_fold_scores = []
cb_fold_thresholds = []
cb_test_folds = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n--- Fold {fold}/{N_FOLDS} ---")

    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]

    model = CatBoostClassifier(**cb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=(X_val, y_val),
        use_best_model=True,
        early_stopping_rounds=200,
        verbose=False,
    )

    cb_models.append(model)
    fold_oof = model.predict_proba(X_val)[:, 1]
    cb_oof[val_idx] = fold_oof

    fold_test = model.predict_proba(X_test)[:, 1]
    cb_test += fold_test / N_FOLDS
    cb_test_folds.append(fold_test)

    best_th, best_f1 = 0.5, 0
    for th in np.arange(0.1, 0.9, 0.01):
        f1 = f1_score(y_val, (fold_oof >= th).astype(int))
        if f1 > best_f1:
            best_f1, best_th = f1, th

    cb_fold_scores.append(best_f1)
    cb_fold_thresholds.append(best_th)
    print(f"Fold {fold}: F1={best_f1:.4f} at threshold={best_th:.2f}")

def neg_f1(th):
    return -f1_score(y, (cb_oof >= th).astype(int))

result = minimize_scalar(neg_f1, bounds=(0.1, 0.9), method="bounded")
cb_best_th = result.x
cb_best_f1 = -result.fun
cb_auc = roc_auc_score(y, cb_oof)

print(f"\nCatBoost CV F1: {np.mean(cb_fold_scores):.4f} ± {np.std(cb_fold_scores):.4f}")
print(f"CatBoost Global F1: {cb_best_f1:.4f} at threshold={cb_best_th:.2f}")
print(f"CatBoost ROC-AUC: {cb_auc:.4f}")

# Submission từ fold có F1 cao nhất
best_cb_fold = int(np.argmax(cb_fold_scores))
best_cb_th = float(cb_fold_thresholds[best_cb_fold])
best_cb_test = np.asarray(cb_test_folds[best_cb_fold], dtype=float)

cb_single_preds = (best_cb_test >= best_cb_th).astype(int)
sub_cb_single = pd.DataFrame({
    "object_id": test_log["object_id"],
    "target": cb_single_preds
})
sub_cb_single.to_csv("submission_single_fold_CAT.csv", index=False)
print(f"\nSaved single-fold CatBoost submission (best fold #{best_cb_fold+1}) -> submission_single_fold_CAT.csv")


TRAINING LIGHTGBM

--- Fold 1/5 ---
Fold 1: F1=0.5263 at threshold=0.19

--- Fold 2/5 ---
Fold 2: F1=0.6250 at threshold=0.30

--- Fold 3/5 ---
Fold 3: F1=0.5067 at threshold=0.12

--- Fold 4/5 ---
Fold 4: F1=0.6667 at threshold=0.34

--- Fold 5/5 ---
Fold 5: F1=0.5758 at threshold=0.35

LightGBM CV F1: 0.5801 ± 0.0597
LightGBM Global F1: 0.5631 at threshold=0.34
LightGBM ROC-AUC: 0.9425

Saved single-fold LGB submission (best fold #4) -> submission_single_fold_LGB.csv


In [ ]:
# ============================================================
# CELL 18: SHAP ANALYSIS FOR WEAKNESS DETECTION
# ============================================================

print("\n" + "="*70)
print("ANALYSIS: SHAP VALUES FOR FEATURE IMPORTANCE (CatBoost)")
print("="*70)

try:
    import shap
    print("✅ SHAP library available\n")
except ImportError:
    print("❌ Installing SHAP...")
    !pip install -q shap
    import shap
    print("✅ SHAP installed\n")

# Use a sample for SHAP (tính toán nhanh hơn)
# Hoặc dùng toàn bộ data nếu có đủ thời gian
SAMPLE_SIZE = min(10000, len(X))  # Sample size for SHAP calculation

print(f"Using sample size: {SAMPLE_SIZE} for SHAP calculation")
print("(For full dataset, increase SAMPLE_SIZE or use tree explainer)\n")

# Sample data
sample_idx = np.random.choice(len(X), size=SAMPLE_SIZE, replace=False)
X_sample = X[sample_idx]
y_sample = y[sample_idx]

# Use one of the trained CatBoost models (best fold)
best_model_idx = np.argmax(cb_fold_scores)
shap_model = cb_models[best_model_idx]

print(f"Using CatBoost model from fold {best_model_idx + 1} (best F1: {cb_fold_scores[best_model_idx]:.4f})\n")

# Calculate SHAP values
print("Calculating SHAP values...")
explainer = shap.TreeExplainer(shap_model)
shap_values = explainer.shap_values(X_sample)

# shap_values is a list: [shap_values_class_0, shap_values_class_1]
shap_values_class1 = shap_values[1] if isinstance(shap_values, list) else shap_values

print(f"✅ SHAP values calculated: shape {shap_values_class1.shape}\n")

# ============================================================
# 1. GLOBAL FEATURE IMPORTANCE (SHAP)
# ============================================================
print("="*70)
print("1. GLOBAL FEATURE IMPORTANCE (SHAP)")
print("="*70)

# Mean absolute SHAP values = global importance
shap_importance = np.abs(shap_values_class1).mean(axis=0)
shap_importance_df = pd.DataFrame({
    'feature': feature_names[:len(shap_importance)],  # Ensure same length
    'shap_importance': shap_importance,
}).sort_values('shap_importance', ascending=False)

print("\nTop 30 Features by SHAP Importance:")
print(shap_importance_df.head(30).to_string(index=False))

print("\nBottom 20 Features by SHAP Importance (potentially useless):")
print(shap_importance_df.tail(20).to_string(index=False))

# ============================================================
# 2. FEATURE IMPACT BY CLASS
# ============================================================
print("\n" + "="*70)
print("2. FEATURE IMPACT BY CLASS (TDE vs Non-TDE)")
print("="*70)

# Separate SHAP values by actual class
tde_shap = shap_values_class1[y_sample == 1]
nontde_shap = shap_values_class1[y_sample == 0]

print(f"\nTDE samples: {len(tde_shap)}, Non-TDE samples: {len(nontde_shap)}")

tde_mean_shap = np.abs(tde_shap).mean(axis=0)
nontde_mean_shap = np.abs(nontde_shap).mean(axis=0)

class_impact_df = pd.DataFrame({
    'feature': feature_names[:len(tde_mean_shap)],
    'shap_TDE': tde_mean_shap,
    'shap_NonTDE': nontde_mean_shap,
    'shap_diff': tde_mean_shap - nontde_mean_shap,
}).sort_values('shap_diff', ascending=False)

print("\nTop 20 Features MORE Important for TDE predictions:")
print(class_impact_df.head(20)[['feature', 'shap_TDE', 'shap_NonTDE', 'shap_diff']].to_string(index=False))

print("\nTop 20 Features MORE Important for Non-TDE predictions:")
print(class_impact_df.tail(20)[['feature', 'shap_TDE', 'shap_NonTDE', 'shap_diff']].to_string(index=False))

# Features with very different importance between classes
class_specific = class_impact_df[np.abs(class_impact_df['shap_diff']) > 0.01].sort_values('shap_diff', key=abs, ascending=False)
print(f"\n   Found {len(class_specific)} features with CLASS-SPECIFIC importance:")
print(class_specific.head(15)[['feature', 'shap_TDE', 'shap_NonTDE', 'shap_diff']].to_string(index=False))
print(f"   → These features behave differently for TDE vs Non-TDE")

# ============================================================
# 3. MISCLASSIFICATION ANALYSIS WITH SHAP
# ============================================================
print("\n" + "="*70)
print("3. MISCLASSIFICATION ANALYSIS WITH SHAP")
print("="*70)

# Get predictions for sample
sample_preds = shap_model.predict_proba(X_sample)[:, 1]
sample_preds_binary = (sample_preds >= cb_fold_thresholds[best_model_idx]).astype(int)
sample_y = y_sample

# Find misclassifications
fp_mask = (sample_preds_binary == 1) & (sample_y == 0)
fn_mask = (sample_preds_binary == 0) & (sample_y == 1)
tp_mask = (sample_preds_binary == 1) & (sample_y == 1)
tn_mask = (sample_preds_binary == 0) & (sample_y == 0)

print(f"\nFP (False Positives): {fp_mask.sum()}")
print(f"FN (False Negatives): {fn_mask.sum()}")
print(f"TP (True Positives): {tp_mask.sum()}")
print(f"TN (True Negatives): {tn_mask.sum()}")

if fp_mask.sum() > 0:
    fp_shap = shap_values_class1[fp_mask]
    tn_shap = shap_values_class1[tn_mask][:min(fp_mask.sum(), tn_mask.sum())]
    
    fp_mean_shap = np.abs(fp_shap).mean(axis=0)
    tn_mean_shap = np.abs(tn_shap).mean(axis=0)
    
    fp_analysis = pd.DataFrame({
        'feature': feature_names[:len(fp_mean_shap)],
        'fp_shap': fp_mean_shap,
        'tn_shap': tn_mean_shap,
        'diff': fp_mean_shap - tn_mean_shap,
    }).sort_values('diff', ascending=False)
    
    print("\n   Top 15 Features causing False Positives (confusing Non-TDE as TDE):")
    print(fp_analysis.head(15)[['feature', 'fp_shap', 'tn_shap', 'diff']].to_string(index=False))

if fn_mask.sum() > 0:
    fn_shap = shap_values_class1[fn_mask]
    tp_shap = shap_values_class1[tp_mask][:min(fn_mask.sum(), tp_mask.sum())]
    
    fn_mean_shap = np.abs(fn_shap).mean(axis=0)
    tp_mean_shap = np.abs(tp_shap).mean(axis=0)
    
    fn_analysis = pd.DataFrame({
        'feature': feature_names[:len(fn_mean_shap)],
        'fn_shap': fn_mean_shap,
        'tp_shap': tp_mean_shap,
        'diff': fn_mean_shap - tp_mean_shap,
    }).sort_values('diff', ascending=False)
    
    print("\n   Top 15 Features causing False Negatives (missing TDE):")
    print(fn_analysis.head(15)[['feature', 'fn_shap', 'tp_shap', 'diff']].to_string(index=False))

# ============================================================
# 4. FEATURE VALUES THAT CONFUSE MODEL
# ============================================================
print("\n" + "="*70)
print("4. FEATURE VALUES THAT CAUSE CONFUSION")
print("="*70)

# For misclassified samples, check if SHAP values are unusually high/low
if fp_mask.sum() > 0 or fn_mask.sum() > 0:
    misclassified_shap = shap_values_class1[fp_mask | fn_mask]
    correctly_classified_shap = shap_values_class1[tp_mask | tn_mask]
    
    if len(misclassified_shap) > 0 and len(correctly_classified_shap) > 0:
        misclassified_mean = misclassified_shap.mean(axis=0)
        correct_mean = correctly_classified_shap.mean(axis=0)
        
        confusion_df = pd.DataFrame({
            'feature': feature_names[:len(misclassified_mean)],
            'misclassified_shap': misclassified_mean,
            'correct_shap': correct_mean,
            'shap_diff': misclassified_mean - correct_mean,
        }).sort_values('shap_diff', key=abs, ascending=False)
        
        print("\n   Features with DIFFERENT SHAP values for misclassified vs correctly classified:")
        print(confusion_df.head(20)[['feature', 'misclassified_shap', 'correct_shap', 'shap_diff']].to_string(index=False))

print("\n" + "="*70)
print("KEY INSIGHTS FROM SHAP")
print("="*70)

insights = []

if len(class_specific) > 10:
    insights.append(f"⚠️  {len(class_specific)} features have class-specific importance")
    insights.append("   → Consider class-specific feature engineering or weighted features")

if fp_mask.sum() > 5 or fn_mask.sum() > 5:
    insights.append(f"⚠️  Misclassifications found: {fp_mask.sum()} FP, {fn_mask.sum()} FN")
    insights.append("   → Check feature values and interactions for these cases")

if len(insights) == 0:
    print("\n✅ SHAP analysis shows consistent feature importance")
else:
    print("\n")
    for i, insight in enumerate(insights, 1):
        print(f"{i}. {insight}")

print("\n" + "="*70)

# Save SHAP importance for later use
shap_importance_df.to_csv(f"{GBM_MODEL_DIR}/shap_importance.csv", index=False)
print(f"\n✅ Saved SHAP importance to {GBM_MODEL_DIR}/shap_importance.csv")


ANALYSIS: SHAP VALUES FOR FEATURE IMPORTANCE
✅ SHAP library available

Using sample size: 3043 for SHAP calculation
(For full dataset, increase SAMPLE_SIZE or use tree explainer)

Using model from fold 4 (best F1: 0.6667)

Calculating SHAP values...
✅ SHAP values calculated: shape (3043, 246)

1. GLOBAL FEATURE IMPORTANCE (SHAP)

Top 30 Features by SHAP Importance:
                feature  shap_importance
i_gp2d_peaks_pos_frac_2         0.488558
    i_gp2d_time_fwd_0.5         0.420239
              i_snr_max         0.355115
    i_gp2d_decline_rate         0.312741
           flux_std_all         0.203499
         flux_range_all         0.187759
           flux_max_all         0.181696
    r_gp2d_time_bwd_0.8         0.167534
      count_max_fall_30         0.159222
             r_flux_std         0.157541
             i_flux_min         0.151830
    g_gp2d_time_fwd_0.5         0.148489
  i_gp2d_positive_width         0.137879
               g_pct_25         0.131938
       g_gp2d_pe

In [ ]:
# ============================================================
# CELL 19: SHAP-BASED FEATURE OPTIMIZATION
# ============================================================

print("\n" + "="*70)
print("SHAP-BASED FEATURE OPTIMIZATION")
print("="*70)

# Load SHAP importance if not in memory
import os
if 'shap_importance_df' not in locals():
    if os.path.exists(f"{GBM_MODEL_DIR}/shap_importance.csv"):
        shap_importance_df = pd.read_csv(f"{GBM_MODEL_DIR}/shap_importance.csv")
        print("✅ Loaded SHAP importance from file")
    else:
        print("❌ SHAP importance not found. Run CELL 18 first!")
        raise FileNotFoundError("Please run CELL 18 (SHAP Analysis) first")

print(f"\nTotal features (after SelectKBest): {len(shap_importance_df)}")

# Get original feature data (before SelectKBest transformation)
# We need to work with original X_df, X_test_df
X_df_orig = train_feat.copy()
X_test_df_orig = test_feat.copy()

if "object_id" in X_df_orig.columns:
    X_df_orig = X_df_orig.drop(columns=["object_id"])
if "object_id" in X_test_df_orig.columns:
    X_test_df_orig = X_test_df_orig.drop(columns=["object_id"])

common_cols_orig = sorted(set(X_df_orig.columns) & set(X_test_df_orig.columns))
X_df_orig = X_df_orig[common_cols_orig]
X_test_df_orig = X_test_df_orig[common_cols_orig]

X_df_orig = X_df_orig.replace([np.inf, -np.inf], np.nan).fillna(0)
X_test_df_orig = X_test_df_orig.replace([np.inf, -np.inf], np.nan).fillna(0)

# ============================================================
# STEP 1: REMOVE USELESS FEATURES (SHAP importance < threshold)
# ============================================================
print("\n" + "-"*70)
print("STEP 1: REMOVE USELESS FEATURES")
print("-"*70)

# Threshold: remove features with SHAP importance < 0.001
SHAP_THRESHOLD = 0.001

useless_features = shap_importance_df[
    shap_importance_df['shap_importance'] < SHAP_THRESHOLD
]['feature'].tolist()

print(f"Features to remove (SHAP < {SHAP_THRESHOLD}): {len(useless_features)}")
if len(useless_features) > 0:
    print(f"  Examples: {useless_features[:10]}")

# Remove from original datasets
X_df_cleaned = X_df_orig.drop(columns=[f for f in useless_features if f in X_df_orig.columns], errors='ignore')
X_test_df_cleaned = X_test_df_orig.drop(columns=[f for f in useless_features if f in X_test_df_orig.columns], errors='ignore')

print(f"\n✅ Removed {len(X_df_orig.columns) - len(X_df_cleaned.columns)} useless features")
print(f"   Remaining features: {len(X_df_cleaned.columns)}")

# ============================================================
# STEP 2: CREATE FEATURE INTERACTIONS FOR TOP FEATURES
# ============================================================
print("\n" + "-"*70)
print("STEP 2: CREATE FEATURE INTERACTIONS FOR TOP FEATURES")
print("-"*70)

# Get top 10 features by SHAP importance
top_shap_features = shap_importance_df.head(10)['feature'].tolist()
print(f"\nTop 10 SHAP features:")
for i, feat in enumerate(top_shap_features, 1):
    shap_val = shap_importance_df[shap_importance_df['feature'] == feat]['shap_importance'].iloc[0]
    print(f"  {i}. {feat}: {shap_val:.4f}")

# Create interactions between top features (multiplicative)
print(f"\nCreating interactions for top i-band features...")
interaction_count = 0

# Focus on i-band features (they're most important)
i_band_features = [f for f in top_shap_features if f.startswith('i_')]
other_top_features = [f for f in top_shap_features if not f.startswith('i_')]

# Create interactions: top i-feature × top other feature
for i_feat in i_band_features[:3]:  # Top 3 i-features
    if i_feat not in X_df_cleaned.columns:
        continue
    
    for other_feat in other_top_features[:3]:  # Top 3 other features
        if other_feat not in X_df_cleaned.columns:
            continue
        
        # Skip self-interaction
        if i_feat == other_feat:
            continue
        
        # Create multiplicative interaction
        interaction_name = f"{i_feat}_x_{other_feat}"
        
        # Normalize before multiplication to avoid extreme values
        i_norm = (X_df_cleaned[i_feat] - X_df_cleaned[i_feat].mean()) / (X_df_cleaned[i_feat].std() + 1e-10)
        other_norm = (X_df_cleaned[other_feat] - X_df_cleaned[other_feat].mean()) / (X_df_cleaned[other_feat].std() + 1e-10)
        
        X_df_cleaned[interaction_name] = i_norm * other_norm
        X_test_df_cleaned[interaction_name] = (
            (X_test_df_cleaned[i_feat] - X_df_cleaned[i_feat].mean()) / (X_df_cleaned[i_feat].std() + 1e-10) *
            (X_test_df_cleaned[other_feat] - X_df_cleaned[other_feat].mean()) / (X_df_cleaned[other_feat].std() + 1e-10)
        )
        interaction_count += 1

print(f"✅ Created {interaction_count} feature interactions")

# ============================================================
# STEP 3: CREATE CLASS-SPECIFIC FEATURES (for TDE)
# ============================================================
print("\n" + "-"*70)
print("STEP 3: CREATE CLASS-SPECIFIC FEATURES")
print("-"*70)

# Features important for TDE predictions (from SHAP analysis)
tde_important_features = [
    'i_gp2d_decline_rate',
    'flux_range_all',
    'i_snr_max',
    'i_gp2d_time_fwd_0.5',
]

print(f"\nCreating weighted features for TDE-important features...")
class_specific_count = 0

for feat in tde_important_features:
    if feat not in X_df_cleaned.columns:
        continue
    
    # Create squared feature (non-linear transformation)
    X_df_cleaned[f"{feat}_squared"] = X_df_cleaned[feat] ** 2
    X_test_df_cleaned[f"{feat}_squared"] = X_test_df_cleaned[feat] ** 2
    
    # Create log feature if positive
    if (X_df_cleaned[feat] > 0).all():
        X_df_cleaned[f"{feat}_log"] = np.log1p(X_df_cleaned[feat])
        X_test_df_cleaned[f"{feat}_log"] = np.log1p(X_test_df_cleaned[feat])
        class_specific_count += 1
    
    class_specific_count += 1

print(f"✅ Created {class_specific_count} class-specific features")

# ============================================================
# STEP 4: CREATE ANTI-FP FEATURES (to reduce False Positives)
# ============================================================
print("\n" + "-"*70)
print("STEP 4: CREATE ANTI-FP FEATURES")
print("-"*70)

# Features that cause False Positives (from SHAP analysis)
fp_features = [
    'i_gp2d_time_fwd_0.5',
    'i_gp2d_decline_rate',
    'flux_range_all',
    'u_gp2d_abs_diff',
]

print(f"\nCreating anti-FP features (to reduce false positives)...")
anti_fp_count = 0

for feat in fp_features:
    if feat not in X_df_cleaned.columns:
        continue
    
    # Create threshold-based feature: is feature value > median?
    median_val = X_df_cleaned[feat].median()
    X_df_cleaned[f"{feat}_high"] = (X_df_cleaned[feat] > median_val).astype(float)
    X_test_df_cleaned[f"{feat}_high"] = (X_test_df_cleaned[feat] > median_val).astype(float)
    
    anti_fp_count += 1

print(f"✅ Created {anti_fp_count} anti-FP features")

# ============================================================
# STEP 5: CREATE ANTI-FN FEATURES (to reduce False Negatives)
# ============================================================
print("\n" + "-"*70)
print("STEP 5: CREATE ANTI-FN FEATURES")
print("-"*70)

# Features important for FN cases (from SHAP analysis)
fn_features = [
    'u_gp2d_abs_diff',
    'i_snr_mean',
    'Z',
]

print(f"\nCreating anti-FN features (to reduce false negatives)...")
anti_fn_count = 0

for feat in fn_features:
    if feat not in X_df_cleaned.columns:
        continue
    
    # Create threshold-based feature
    median_val = X_df_cleaned[feat].median()
    X_df_cleaned[f"{feat}_low"] = (X_df_cleaned[feat] < median_val).astype(float)
    X_test_df_cleaned[f"{feat}_low"] = (X_test_df_cleaned[feat] < median_val).astype(float)
    
    anti_fn_count += 1

print(f"✅ Created {anti_fn_count} anti-FN features")

# ============================================================
# STEP 6: FINAL CLEANUP
# ============================================================
print("\n" + "-"*70)
print("STEP 6: FINAL CLEANUP")
print("-"*70)

# Ensure same columns
common_cols = sorted(set(X_df_cleaned.columns) & set(X_test_df_cleaned.columns))
X_df_cleaned = X_df_cleaned[common_cols]
X_test_df_cleaned = X_test_df_cleaned[common_cols]

print(f"\nFinal feature count: {len(common_cols)}")
print(f"  - Original: {len(X_df_orig.columns)}")
print(f"  - Removed useless: {len(useless_features)}")
print(f"  - Added (interactions + transforms): {len(common_cols) - (len(X_df_orig.columns) - len([f for f in useless_features if f in X_df_orig.columns]))}")

# Replace NaN/Inf
X_df_cleaned = X_df_cleaned.replace([np.inf, -np.inf], np.nan).fillna(0)
X_test_df_cleaned = X_test_df_cleaned.replace([np.inf, -np.inf], np.nan).fillna(0)

# Update feature names and data
feature_names_optimized = X_df_cleaned.columns.tolist()
X_raw_optimized = X_df_cleaned.values
X_test_raw_optimized = X_test_df_cleaned.values

print(f"\n✅ Feature optimization completed!")
print(f"   Optimized shape: Train {X_raw_optimized.shape}, Test {X_test_raw_optimized.shape}")

# Save optimized feature names
joblib.dump(feature_names_optimized, f"{GBM_MODEL_DIR}/optimized_feature_names.pkl")
print(f"✅ Saved optimized feature names")


SHAP-BASED FEATURE OPTIMIZATION

Total features (after SelectKBest): 246

----------------------------------------------------------------------
STEP 1: REMOVE USELESS FEATURES
----------------------------------------------------------------------
Features to remove (SHAP < 0.001): 24
  Examples: ['y_flux_min', 'gp2d_peak_dt_r_z', 'i_gp2d_time_bwd_0.5', 'i_gp2d_time_bwd_0.8', 'snr_median', 'r_gp2d_peaks_pos_count', 'i_gp2d_negative_width', 'i_gp2d_rise_decline_ratio', 'y_flux_mean', 'u_gp2d_fwhm']

✅ Removed 24 useless features
   Remaining features: 284

----------------------------------------------------------------------
STEP 2: CREATE FEATURE INTERACTIONS FOR TOP FEATURES
----------------------------------------------------------------------

Top 10 SHAP features:
  1. i_gp2d_peaks_pos_frac_2: 0.4886
  2. i_gp2d_time_fwd_0.5: 0.4202
  3. i_snr_max: 0.3551
  4. i_gp2d_decline_rate: 0.3127
  5. flux_std_all: 0.2035
  6. flux_range_all: 0.1878
  7. flux_max_all: 0.1817
  8. r_gp2d_t